## IndexとType
Elasticsearchに実際にデータを入れていく前にIndexとTypeの違いをまとめます。

RDBになぞらえてシンプルに

+ Index=DB
+ Type=Table

と説明されることがあるらしいですが、正確には違うらしいです。

とりあえずはそんな感じの認識で進めます。

進めてええんかい（ツッコミ）

細かい意味は以下のページを参考にしてください

- [Index vs. Type | Elastic](https://www.elastic.co/jp/blog/index-vs-type)
- [データ構造について – AWSで始めるElasticSearch(4) ｜ Developers.IO](https://dev.classmethod.jp/cloud/aws/use-elasticsearch-4-data-structure/)

## PythonからElasticsearchを利用してみる

まあ、インストール

In [1]:
! pip install elasticsearch

### Index():ドキュメントの追加、更新
適当にスニペットを書いていきます
```Python
es = Elasticsearch()
```
で特にオプションを記述しなければ、localhost:9200を見に行きます

index()メソッドではドキュメントの追加、更新ができます。

この時点で指定したインデックス、タイプが無ければ自動で作成されます。

pythonで読む前にサーバは起動しておく必要があるみたい

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch()

es.index(index="my-index", doc_type="test-type", body={"key": "value"})

/home/naoaki/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'my-index',
 '_type': 'test-type',
 '_id': 'jVZggHgBtsqmUzTO1MsR',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

これでmy-indexというインデックスにtest-typeというタイプで{"key": "value"}というドキュメントが作成されました

idを指定してドキュメントを作成することも可能

In [3]:
es.index(index="my-index", doc_type="test-type", id=1,  body={"key": "value1"})

{'_index': 'my-index',
 '_type': 'test-type',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

### get():ドキュメントを取得する
このようにインデックス、タイプ、id、を指定することで登録済みのドキュメントを取得できる

In [4]:
es.get(index="my-index", doc_type="test-type", id=1)

/home/naoaki/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'my-index',
 '_type': 'test-type',
 '_id': '1',
 '_version': 1,
 '_seq_no': 1,
 '_primary_term': 1,
 'found': True,
 '_source': {'key': 'value1'}}

### search():ドキュメントの検索
適当にドキュメントを三つぐらい追加しておきます

In [6]:
es.index(index="my-index", doc_type="test-type", id=1, body={"key":"value1"})
es.index(index="my-index", doc_type="test-type", id=2, body={"key":"value2"})
es.index(index="my-index", doc_type="test-type", id=3, body={"key":"value3"})

{'_index': 'my-index',
 '_type': 'test-type',
 '_id': '3',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 5,
 '_primary_term': 1}

検索するためにはインデックスとクエリを指定します

クエリにmatch_allとすれば全権取得できます

In [7]:
es.search(index="my-index", body={"query": {"match_all": {}}})

{'took': 2287,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'my-index',
    '_type': 'test-type',
    '_id': 'jVZggHgBtsqmUzTO1MsR',
    '_score': 1.0,
    '_source': {'key': 'value'}},
   {'_index': 'my-index',
    '_type': 'test-type',
    '_id': '1',
    '_score': 1.0,
    '_source': {'key': 'value1'}},
   {'_index': 'my-index',
    '_type': 'test-type',
    '_id': '2',
    '_score': 1.0,
    '_source': {'key': 'value2'}},
   {'_index': 'my-index',
    '_type': 'test-type',
    '_id': '3',
    '_score': 1.0,
    '_source': {'key': 'value3'}}]}}

このように条件を付ければフィルターをかけることができます

In [8]:
es.search(index="my-index", body={"query": {"match": {"key":"value1"}}})

{'took': 65,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.87546873,
  'hits': [{'_index': 'my-index',
    '_type': 'test-type',
    '_id': '1',
    '_score': 0.87546873,
    '_source': {'key': 'value1'}}]}}